In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap, cumsum
from bokeh.layouts import row, column, gridplot
from bokeh.models import DatetimeTickFormatter
from bokeh.models import NumeralTickFormatter

In [2]:
#load data
new_dataset = pd.read_csv('archive/consolidate_data.csv')
latest_std_cost = pd.read_csv('archive/latest_std_cost.csv')
del latest_std_cost['Date']
#append latest standard cost to database
process_dataset=new_dataset.merge(latest_std_cost,on='ItemId',how='left')
#check if any latest_std_cost is missing. If yes fill with original CostPerUnit data
process_dataset['Std_cost_latest'].fillna(process_dataset['CostPerUnit'],inplace=True)
#calculate stock value = soh * standard cost
process_dataset ['Stock_value'] = process_dataset['SOH'] * process_dataset['Std_cost_latest']
#subset data and update date to datetime type
process_dataset = process_dataset [['Date','Stock_value','ItemId','VendorCode','VendorName']]
process_dataset['Date'] = pd.to_datetime(process_dataset['Date'],format='%d-%m-%Y') 
process_dataset = process_dataset.sort_values('Date',ascending=True)
 
##### plot by item
#link between pd and bokeh for original data
src1 = ColumnDataSource(process_dataset)


# create CDS for empty dataframe
filteredSource = ColumnDataSource(data=dict(Date=[],Stock_value=[]))


p = figure(title="Invetory level",
                x_axis_type="datetime",
                x_axis_label='Date',
                y_axis_label='Stock_value')


item_draw =p.line(x='Date',y='Stock_value',source = filteredSource,legend_label="Stock level",hover_color = 'red')

#format y axis
p.left[0].formatter.use_scientific = False
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,000")
#format x axis
p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d/%m/%Y"],
    )

"""
#add hovertool
p.add_tools(HoverTool(
    tooltips=[
        ( "Date","$x{%F}"),
        ( "Stock_value", "$y{"+f"0.00"+" a}" )
    ],
    formatters={
        '$x' : 'datetime',
    },
))
"""
p.legend.location = 'bottom_right'

item_data_list = sorted(process_dataset['ItemId'].unique().tolist())
#get list of item for box
item_list= ['Please choose item...'] + item_data_list

select = Select(title='Items', value=item_list[0], options=item_list)

callback = CustomJS(
    args=dict(src1=src1,filteredSource=filteredSource,select=select),
    code= """
    const data = src1.data;
    const item_filtered = select.value;
    const plotdata = filteredSource.data;
    plotdata['Stock_value']=[];
    plotdata['Date']=[];
    
    for (var i=0; i<data['ItemId'].length; i++) {
        if(data['ItemId'][i] == item_filtered ) {
            plotdata['Stock_value'].push(data['Stock_value'][i])
            plotdata['Date'].push(data['Date'][i])
        }
    }
    filteredSource.change.emit();
    """
)
select.js_on_change('value',callback)


layout = row(p,column(select))
show(layout)